In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import numpy as np
import json
import io
import re
import requests
from collections import Counter
from tqdm import tqdm
import gc
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports loaded")

✅ Imports loaded


In [0]:

storage_account = "lab94290"  
container = "airbnb"
sas_token="sp=rle&st=2025-12-24T17:37:04Z&se=2026-02-28T01:52:04Z&spr=https&sv=2024-11-04&sr=c&sig=a0lx%2BS6PuS%2FvJ9Tbt4NKdCJHLE9d1Y1D6vpE1WKFQtk%3D"
sas_token = sas_token.lstrip('?')
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)
path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/airbnb_1_12_parquet"

print("📥 Loading data from Azure...")
df_spark = spark.read.parquet(path)
print(f"   Total rows in Spark: {df_spark.count():,}")

# Load sample_data first
sample_data = pd.read_csv('sample_data.csv')  # Adjust path if needed

# Get property IDs
property_col = 'property_id'  # Adjust if different column name
sample_property_ids = sample_data[property_col].astype(str).tolist()

print(f"📋 Filtering to {len(sample_property_ids)} properties from sample_data...")

df_spark_filtered = df_spark.filter(df_spark['property_id'].isin(sample_property_ids))

print(f"   Filtered rows: {df_spark_filtered.count():,}")

df = df_spark_filtered.toPandas()

print(f"✅ Converted to Pandas: {len(df):,} rows")

📥 Loading data from Azure...
   Total rows in Spark: 2,098,880
📋 Filtering to 2546 properties from sample_data...
   Filtered rows: 2,574
✅ Converted to Pandas: 2,574 rows


In [0]:
# Load your sample data
sample_data = pd.read_csv('sample_data.csv')  

print(f"📋 Sample data loaded: {len(sample_data)} rows")
print(f"   Columns: {list(sample_data.columns)}")

# Show first few rows to identify the property_id column
display(sample_data.head())

📋 Sample data loaded: 2546 rows
   Columns: ['city', 'price', 'lat', 'long', 'guests', 'pets_allowed', 'property_id', 'is_supperhost', 'host_number_of_reviews', 'host_rating', 'hosts_year', 'host_response_rate', 'is_guest_favorite', 'property_number_of_reviews', 'seller_id', 'state', 'country', 'num_images', 'cancellation_type', 'log_price', 'bedrooms', 'beds', 'bathrooms', 'property_type', 'has_ac', 'has_wifi', 'has_pool', 'has_kitchen', 'has_parking', 'amenities_count', 'poi_restaurants', 'poi_cafes', 'poi_transit', 'poi_bus_stops', 'poi_supermarkets', 'poi_total', 'data_source_type', 'walk_score', 'booking_hotel_count', 'booking_avg_rating', 'booking_total_reviews', 'booking_data_source', 'beds_per_bedroom', 'guests_per_bedroom', 'bathrooms_per_bedroom', 'host_quality_score', 'reviews_per_year', 'amenities_per_guest', 'premium_amenity_score', 'tourism_density', 'local_hotel_competition_quality', 'foodie_score', 'transport_score', 'cancellation_type_indexed', 'property_type_indexed',

city,price,lat,long,guests,pets_allowed,property_id,is_supperhost,host_number_of_reviews,host_rating,hosts_year,host_response_rate,is_guest_favorite,property_number_of_reviews,seller_id,state,country,num_images,cancellation_type,log_price,bedrooms,beds,bathrooms,property_type,has_ac,has_wifi,has_pool,has_kitchen,has_parking,amenities_count,poi_restaurants,poi_cafes,poi_transit,poi_bus_stops,poi_supermarkets,poi_total,data_source_type,walk_score,booking_hotel_count,booking_avg_rating,booking_total_reviews,booking_data_source,beds_per_bedroom,guests_per_bedroom,bathrooms_per_bedroom,host_quality_score,reviews_per_year,amenities_per_guest,premium_amenity_score,tourism_density,local_hotel_competition_quality,foodie_score,transport_score,cancellation_type_indexed,property_type_indexed,data_source_type_indexed,features,prediction
Paris,104.97,48.85555,2.38044,2,0,19375975,0,116,4.68,11,100,0,113,11697538,Ile-de-France,France,8,No refund,4.663156035186745,2,2,1.0,entire_place,0,1,0,1,0,16,45.54838709677419,14.5,8.838709677419354,7.258064516129032,1.064516129032258,77.20967741935483,city_fallback,75.7,52,7.177272727272728,10352,exact_grid,1,1.0,0.5,0.6744,10.272727272727272,8.0,0,3.970291913552122,66.35411324990315,60.04838709677419,16.096774193548384,1,0,0,"{""vectorType"":""dense"",""length"":45,""values"":[48.85555,2.38044,2,0,0,116,4.68,11,100,0,113,8,2,2,1,0,1,0,1,0,16,45.54838709677419,14.5,8.838709677419354,7.258064516129032,1.064516129032258,77.20967741935483,75.7,52,7.177272727272728,10352,1,1,0.5,0.6744,10.272727272727273,8,0,3.970291913552122,66.35411324990315,60.04838709677419,16.096774193548384,1,0,0]}",347.76150448170097
Paris,133.31,48.85667200140991,2.377552770197512,2,0,983907471264009659,0,9,4.67,2,100,0,9,118573158,Ile-de-France,France,17,No refund,4.900150560920984,1,1,1.0,entire_place,1,1,0,1,0,8,45.54838709677419,14.5,8.838709677419354,7.258064516129032,1.064516129032258,77.20967741935483,city_fallback,75.3,78,7.145714285714287,5310,exact_grid,1,2.0,1.0,0.6736,4.5,4.0,1,4.3694478524670215,61.29261738993263,60.04838709677419,16.096774193548384,1,0,0,"{""vectorType"":""dense"",""length"":45,""values"":[48.85667200140991,2.377552770197512,2,0,0,9,4.67,2,100,0,9,17,1,1,1,1,1,0,1,0,8,45.54838709677419,14.5,8.838709677419354,7.258064516129032,1.064516129032258,77.20967741935483,75.3,78,7.145714285714287,5310,1,2,1,0.6736,4.5,4,1,4.3694478524670215,61.29261738993263,60.04838709677419,16.096774193548384,1,0,0]}",435.0680166672462
Paris,125.35,48.8503,2.37036,2,1,53885026,1,31,4.94,3,100,0,31,130553618,Ile-de-France,France,4,Partial refund,4.839055833834204,1,1,1.0,entire_place,0,1,0,1,1,32,0.0,0.0,0.0,0.0,0.0,0.0,exact_grid,1.8,77,7.2073170731707314,2549,exact_grid,1,2.0,1.0,0.9952,10.333333333333334,16.0,1,4.356708826689592,56.5331042091233,0.0,0.0,2,0,1,"{""vectorType"":""dense"",""length"":45,""values"":[48.8503,2.37036,2,1,1,31,4.94,3,100,0,31,4,1,1,1,0,1,0,1,1,32,0,0,0,0,0,0,1.8,77,7.2073170731707314,2549,1,2,1,0.9952000000000001,10.333333333333334,16,1,4.356708826689592,56.5331042091233,0,0,2,0,1]}",213.6674600938227
Paris,615.06,48.865,2.3722,4,0,1067416961927941819,0,8,4.63,1,100,0,8,162419681,Ile-de-France,France,16,Full refund,6.423344361387597,1,1,1.0,entire_place,0,1,0,1,0,4,45.54838709677419,14.5,8.838709677419354,7.258064516129032,1.064516129032258,77.20967741935483,city_fallback,74.5,52,7.837037037037038,4601,exact_grid,1,4.0,1.0,0.6704,8.0,1.0,0,3.970291913552122,66.09950040214697,60.04838709677419,16.096774193548384,0,0,0,"{""vectorType"":""dense"",""length"":45,""values"":[48.865,2.3722,4,0,0,8,4.63,1,100,0,8,16,1,1,1,0,1,0,1,0,4,45.54838709677419,14.5,8.838709677419354,7.258064516129032,1.064516129032258,77.20967741935483,74.5,52,7.837037037037038,4601,1,4,1,0.6704,8,1,0,3.970291913552122,66.09950040214697,60.04838709677419,16.096774193548384,0,0,0]}",515.8223772067678
Paris,152.83,48.86862,2.35268,2,1,554804900919538096,0,48,4.94,3,57,0,48,177030908,Ile-de-France,France,14,Full refund,5.03584809656766

In [0]:
def parse_reviews_text(reviews_str):
    """Extract list of review texts from JSON string"""
    if pd.isna(reviews_str): 
        return []
    try:
        reviews = json.loads(reviews_str)
        if isinstance(reviews, list):
            return [r.replace('<br/>', ' ').strip() for r in reviews if r]
        return []
    except: 
        return []

def parse_category_ratings(rating_str):
    """Extract ratings dictionary from JSON string"""
    if pd.isna(rating_str): 
        return {}
    try:
        ratings = json.loads(rating_str) if isinstance(rating_str, str) else rating_str
        if isinstance(ratings, list):
            return {item['name']: float(item['value']) for item in ratings if 'name' in item}
        return {}
    except: 
        return {}

def extract_host_id(seller_info):
    """Extract host ID from seller_info JSON"""
    try:
        if pd.isna(seller_info): 
            return 'unknown'
        data = json.loads(seller_info) if isinstance(seller_info, str) else seller_info
        return str(data.get('seller_id', 'unknown'))
    except: 
        return 'unknown'

print("✅ Parse functions defined")

✅ Parse functions defined


In [0]:
print("🔄 Processing data...")

# Parse all columns
df['host_id'] = df['seller_info'].apply(extract_host_id)
df['reviews_list'] = df['reviews'].apply(parse_reviews_text)
df['ratings_dict'] = df['category_rating'].apply(parse_category_ratings)
df['review_count'] = df['reviews_list'].apply(len)

# Extract individual ratings
df['cleanliness_rating'] = df['ratings_dict'].apply(lambda x: x.get('Cleanliness'))
df['accuracy_rating'] = df['ratings_dict'].apply(lambda x: x.get('Accuracy'))
df['checkin_rating'] = df['ratings_dict'].apply(lambda x: x.get('Check-in'))
df['communication_rating'] = df['ratings_dict'].apply(lambda x: x.get('Communication'))
df['location_rating'] = df['ratings_dict'].apply(lambda x: x.get('Location'))
df['value_rating'] = df['ratings_dict'].apply(lambda x: x.get('Value'))

print("✅ Data parsed")


# Identify property_id column in sample_data (try common names)
property_col = None
for col in ['property_id', 'Property_ID', 'PropertyID', 'id', 'ID', 'listing_id', 'Listing_ID']:
    if col in sample_data.columns:
        property_col = col
        break

if property_col is None:
    print(f"⚠️ Could not find property_id column. Available: {list(sample_data.columns)}")
    print("   Please set property_col manually in the next line")
    property_col = sample_data.columns[0]  # Default to first column

print(f"📍 Using column '{property_col}' as property identifier")

# Get property IDs from sample
sample_property_ids = set(sample_data[property_col].astype(str).tolist())
print(f"   Sample contains {len(sample_property_ids)} property IDs")

# Filter main dataframe
df_filtered = df[df['property_id'].astype(str).isin(sample_property_ids)].copy()

# Keep only properties with reviews
df_filtered = df_filtered[df_filtered['review_count'] > 0]

print(f"\n✅ FILTERED RESULTS:")
print(f"   Properties with reviews: {len(df_filtered):,}")
print(f"   Total reviews: {df_filtered['review_count'].sum():,}")
print(f"   Unique hosts: {df_filtered['host_id'].nunique():,}")

🔄 Processing data...
✅ Data parsed
📍 Using column 'property_id' as property identifier
   Sample contains 2368 property IDs

✅ FILTERED RESULTS:
   Properties with reviews: 2,448
   Total reviews: 41,617
   Unique hosts: 1,885


In [0]:
print("🔄 Exploding reviews...")

# Keep relevant columns and explode
reviews_exploded = df_filtered[[
    'property_id', 'listing_name', 'host_id', 'location',
    'reviews_list', 'cleanliness_rating', 'accuracy_rating', 'checkin_rating',
    'communication_rating', 'location_rating', 'value_rating'
]].explode('reviews_list').reset_index(drop=True)

# Rename and clean
reviews_exploded = reviews_exploded.rename(columns={'reviews_list': 'review_text'})
reviews_exploded = reviews_exploded[
    reviews_exploded['review_text'].notna() & 
    (reviews_exploded['review_text'].str.len() > 10)
].reset_index(drop=True)

reviews_exploded['review_id'] = range(len(reviews_exploded))

print(f"✅ Exploded to {len(reviews_exploded):,} individual reviews")
print(f"   From {reviews_exploded['property_id'].nunique():,} properties")
print(f"   From {reviews_exploded['host_id'].nunique():,} hosts")

🔄 Exploding reviews...
✅ Exploded to 41,225 individual reviews
   From 2,246 properties
   From 1,884 hosts


In [0]:
from transformers import pipeline

print("📦 Loading sentiment model (DistilBERT)...")
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=-1,  # CPU
    truncation=True,
    max_length=256
)
print("✅ Sentiment model loaded!")

2026-01-31 09:48:22.738980: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-31 09:48:22.934749: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-31 09:48:23.844127: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-31 09:48:30.410824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


📦 Loading sentiment model (DistilBERT)...


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

✅ Sentiment model loaded!


In [0]:
def analyze_sentiment_safe(texts, batch_size=32):
    """Analyze sentiment with memory management"""
    results = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Analyzing sentiment"):
        batch = [str(t)[:256] for t in texts[i:i+batch_size]]
        
        try:
            batch_results = sentiment_analyzer(batch)
            for r in batch_results:
                score = r['score'] if r['label'] == 'POSITIVE' else -r['score']
                results.append({'label': r['label'], 'score': round(score, 4)})
        except:
            for _ in batch:
                results.append({'label': 'NEUTRAL', 'score': 0.0})
        
        # Memory cleanup every 500 batches
        if (i // batch_size) % 500 == 0:
            gc.collect()
    
    return results

# Run sentiment analysis
print(f"🔄 Analyzing sentiment for {len(reviews_exploded):,} reviews...")
print(f"   Estimated time: ~{len(reviews_exploded) // 32 * 1.3 / 60:.0f} minutes")

results = analyze_sentiment_safe(reviews_exploded['review_text'].tolist(), batch_size=32)

reviews_exploded['sentiment_label'] = [r['label'] for r in results]
reviews_exploded['sentiment_score'] = [r['score'] for r in results]

print(f"\n✅ Sentiment analysis complete!")
print(f"\n📊 SENTIMENT DISTRIBUTION:")
print(reviews_exploded['sentiment_label'].value_counts())
print(f"\n📈 Average sentiment score: {reviews_exploded['sentiment_score'].mean():.3f}")

🔄 Analyzing sentiment for 41,225 reviews...
   Estimated time: ~28 minutes


Analyzing sentiment:  27%|██▋       | 348/1289 [09:25<23:

*** WARNING: max output size exceeded, skipping output. ***

Analyzing sentiment: 100%|██████████| 1289/1289 [39:51<00:00,  1.86s/it]


✅ Sentiment analysis complete!

📊 SENTIMENT DISTRIBUTION:
POSITIVE    38680
NEGATIVE     2545
Name: sentiment_label, dtype: int64

📈 Average sentiment score: 0.876


In [0]:
ISSUE_PATTERNS = {
    # ============ NEGATIVE ISSUES ============
    "wifi_issues": [r"\bwifi slow\b", r"\bwifi didn'?t work\b", r"\bno wifi\b", r"\binternet slow\b", r"\bwifi bad\b", r"\bweak wifi\b", r"\bpoor wifi\b", r"\bno internet\b"],
    "dirty_general": [r"\bdirty\b", r"\bnot clean\b", r"\bunclean\b", r"\bfilthy\b", r"\bdusty\b", r"\bneeds cleaning\b"],
    "dirty_bathroom": [r"\bdirty bathroom\b", r"\bbathroom not clean\b", r"\bmold\b", r"\bmildew\b", r"\btoilet dirty\b"],
    "dirty_kitchen": [r"\bdirty kitchen\b", r"\bdirty dishes\b", r"\bkitchen not clean\b", r"\bgreasy\b"],
    "dirty_bedding": [r"\bdirty sheets\b", r"\bstained sheets\b", r"\bdirty towels\b", r"\bstained towels\b"],
    "bugs_pests": [r"\bbugs\b", r"\binsects\b", r"\bants\b", r"\bcockroach", r"\broaches\b", r"\bmice\b", r"\bbed bugs\b"],
    "smell_issues": [r"\bbad smell\b", r"\bsmelled bad\b", r"\bodor\b", r"\bmusty\b", r"\bsmoke smell\b", r"\bsmelly\b"],
    "noise_general": [r"\bnoisy\b", r"\btoo loud\b", r"\bnot quiet\b"],
    "noise_street": [r"\bstreet noise\b", r"\btraffic noise\b", r"\bnoisy street\b", r"\bloud traffic\b"],
    "noise_neighbors": [r"\bloud neighbors\b", r"\bnoisy neighbors\b", r"\bthin walls\b", r"\bcould hear neighbors\b"],
    "too_hot": [r"\btoo hot\b", r"\bno ac\b", r"\bac didn'?t work\b", r"\bac broken\b", r"\bno air conditioning\b"],
    "too_cold": [r"\btoo cold\b", r"\bno heating\b", r"\bfreezing\b", r"\bheater broken\b", r"\bno heat\b"],
    "uncomfortable_bed": [r"\buncomfortable bed\b", r"\bbad mattress\b", r"\bhard mattress\b", r"\blumpy mattress\b"],
    "hot_water_issues": [r"\bno hot water\b", r"\bcold water only\b", r"\bwater pressure low\b", r"\bhot water ran out\b"],
    "plumbing_issues": [r"\btoilet broken\b", r"\bclogged drain\b", r"\bsink clogged\b", r"\bleaking\b", r"\bdripping\b"],
    "checkin_problems": [r"\bcheck-?in problem\b", r"\bcheck-?in difficult\b", r"\bcouldn'?t check in\b"],
    "hard_to_find": [r"\bhard to find\b", r"\bcouldn'?t find\b", r"\bwrong address\b", r"\bconfusing directions\b", r"\bgot lost\b"],
    "key_issues": [r"\bkey didn'?t work\b", r"\blockbox problem\b", r"\bcouldn'?t get in\b", r"\blocked out\b", r"\bwrong code\b"],
    "host_unresponsive": [r"\bhost didn'?t respond\b", r"\bno response\b", r"\bslow response\b", r"\bcouldn'?t reach host\b"],
    "overpriced": [r"\boverpriced\b", r"\btoo expensive\b", r"\bnot worth the price\b", r"\bnot worth\b"],
    "too_small": [r"\btoo small\b", r"\btiny\b", r"\bcramped\b", r"\bsmaller than expected\b"],
    "safety_concerns": [r"\bdidn'?t feel safe\b", r"\bunsafe\b", r"\bsketchy area\b", r"\bdangerous\b"],
    
    # ============ POSITIVE THEMES ============
    "great_location": [r"\bgreat location\b", r"\bperfect location\b", r"\bamazing location\b", r"\bbest location\b", r"\bexcellent location\b"],
    "walkable": [r"\bwalking distance\b", r"\bwalkable\b", r"\bclose to everything\b", r"\bsteps away\b", r"\beasy walk\b"],
    "quiet_area": [r"\bquiet area\b", r"\bpeaceful\b", r"\bquiet neighborhood\b", r"\bvery quiet\b", r"\btranquil\b"],
    "central": [r"\bcentral location\b", r"\bin the center\b", r"\bheart of the city\b", r"\bdowntown\b", r"\bcity center\b"],
    "very_clean": [r"\bvery clean\b", r"\bspotless\b", r"\bimmaculate\b", r"\bsuper clean\b", r"\bsparkling clean\b", r"\bpristine\b"],
    "great_host": [r"\bgreat host\b", r"\bamazing host\b", r"\bwonderful host\b", r"\bbest host\b", r"\bfantastic host\b", r"\blovely host\b"],
    "responsive_host": [r"\bresponsive\b", r"\bquick response\b", r"\bfast response\b", r"\bresponded quickly\b", r"\bimmediate response\b"],
    "friendly_host": [r"\bfriendly host\b", r"\bwelcoming host\b", r"\bwarm welcome\b", r"\bhost was friendly\b", r"\bhost was kind\b"],
    "great_communication": [r"\bgreat communication\b", r"\beasy communication\b", r"\bclear communication\b", r"\bgood communication\b"],
    "great_view": [r"\bgreat view\b", r"\bamazing view\b", r"\bbeautiful view\b", r"\bstunning view\b", r"\bincredible view\b"],
    "beautiful_place": [r"\bbeautiful place\b", r"\bgorgeous\b", r"\bstunning\b", r"\blovely decor\b", r"\bbeautifully decorated\b", r"\bcharming\b"],
    "good_value": [r"\bgreat value\b", r"\bgood value\b", r"\bworth the price\b", r"\bgood price\b", r"\bexcellent value\b", r"\baffordable\b"],
    "well_equipped": [r"\bwell equipped\b", r"\bhad everything\b", r"\bfully equipped\b", r"\ball amenities\b", r"\beverything we needed\b"],
    "great_kitchen": [r"\bgreat kitchen\b", r"\bfull kitchen\b", r"\bwell equipped kitchen\b", r"\bkitchen had everything\b"],
    "good_bed": [r"\bcomfortable bed\b", r"\bgreat bed\b", r"\bslept well\b", r"\bcomfy bed\b", r"\bgood mattress\b"],
    "good_wifi": [r"\bfast wifi\b", r"\bgreat wifi\b", r"\bwifi worked\b", r"\bstrong wifi\b", r"\bgood internet\b"],
    "very_comfortable": [r"\bvery comfortable\b", r"\bsuper comfortable\b", r"\bso comfortable\b", r"\bcozy\b", r"\bhomey\b", r"\bfelt like home\b"],
    "spacious": [r"\bspacious\b", r"\blots of space\b", r"\bplenty of room\b", r"\broomy\b", r"\bbig apartment\b"],
    "will_return": [r"\bwill be back\b", r"\bcome back\b", r"\breturn\b", r"\bstay again\b", r"\bbook again\b", r"\bdefinitely return\b"],
    "highly_recommend": [r"\bhighly recommend\b", r"\bdefinitely recommend\b", r"\bwould recommend\b", r"\bstrongly recommend\b"],
    "exceeded_expectations": [r"\bexceeded expectations\b", r"\bbetter than expected\b", r"\babove and beyond\b", r"\bblown away\b"],
    "perfect_stay": [r"\bperfect stay\b", r"\bamazing stay\b", r"\bwonderful stay\b", r"\bfantastic stay\b", r"\bbest stay\b"],
    "easy_checkin": [r"\beasy check-?in\b", r"\bsmooth check-?in\b", r"\bsimple check-?in\b", r"\bself check-?in\b", r"\bseamless\b"],
}

# List of negative themes (for classification)
NEGATIVE_THEMES = [
    "wifi_issues", "dirty_general", "dirty_bathroom", "dirty_kitchen", "dirty_bedding",
    "bugs_pests", "smell_issues", "noise_general", "noise_street", "noise_neighbors",
    "too_hot", "too_cold", "uncomfortable_bed", "hot_water_issues", "plumbing_issues",
    "checkin_problems", "hard_to_find", "key_issues", "host_unresponsive", "overpriced",
    "too_small", "safety_concerns"
]

print(f"✅ Defined {len(ISSUE_PATTERNS)} theme patterns")
print(f"   Negative: {len(NEGATIVE_THEMES)}")
print(f"   Positive: {len(ISSUE_PATTERNS) - len(NEGATIVE_THEMES)}")

✅ Defined 45 theme patterns
   Negative: 22
   Positive: 23


In [0]:
def extract_themes_regex(text):
    """Extract all themes mentioned in a review using regex"""
    if pd.isna(text):
        return []
    text_lower = text.lower()
    found = []
    for theme, patterns in ISSUE_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, text_lower):
                found.append(theme)
                break  # Only count each theme once per review
    return found

# Apply theme extraction
print("🔍 Extracting themes from reviews...")
reviews_exploded['themes'] = reviews_exploded['review_text'].apply(extract_themes_regex)
reviews_exploded['negative_themes'] = reviews_exploded['themes'].apply(lambda x: [t for t in x if t in NEGATIVE_THEMES])
reviews_exploded['positive_themes'] = reviews_exploded['themes'].apply(lambda x: [t for t in x if t not in NEGATIVE_THEMES])

# Summary
all_themes = [t for themes in reviews_exploded['themes'] for t in themes]
theme_counts = Counter(all_themes)

print(f"\n✅ Theme extraction complete!")
print(f"\n📊 TOP 10 THEMES:")
for theme, count in theme_counts.most_common(10):
    theme_type = "❌" if theme in NEGATIVE_THEMES else "✅"
    print(f"   {theme_type} {theme.replace('_', ' ').title()}: {count}")

🔍 Extracting themes from reviews...

✅ Theme extraction complete!

📊 TOP 10 THEMES:
   ✅ Great Location: 7029
   ✅ Responsive Host: 5850
   ✅ Highly Recommend: 5512
   ✅ Very Comfortable: 4848
   ✅ Walkable: 4548
   ✅ Will Return: 4417
   ✅ Spacious: 3232
   ✅ Very Clean: 3122
   ✅ Well Equipped: 3066
   ✅ Quiet Area: 2543


In [0]:
print("📚 Loading Airbnb knowledge base...")

# Load from file
with open('scraping_results_full_english.json', 'r') as f:
    knowledge_base_raw = json.load(f)

# Extract tips from articles
all_tips = []
for article in knowledge_base_raw:
    if article.get("status") != "success":
        continue
    
    content = article.get("content", {})
    title = content.get("page_title", "")
    
    # Extract from list items
    for item in content.get("list_items", []):
        if 30 < len(item) < 600:
            skip_words = ['privacy', 'footer', 'help center', 'careers', 'newsroom', 'investors']
            if not any(skip in item.lower() for skip in skip_words):
                all_tips.append({'tip': item, 'source': title})
    
    # Extract from paragraphs
    for para in content.get("paragraphs", []):
        if 50 < len(para) < 500:
            skip_words = ['privacy', 'footer', 'help center']
            if not any(skip in para.lower() for skip in skip_words):
                all_tips.append({'tip': para, 'source': title})

print(f"✅ Loaded {len(all_tips)} tips from {len([a for a in knowledge_base_raw if a.get('status') == 'success'])} articles")

📚 Loading Airbnb knowledge base...
✅ Loaded 1251 tips from 73 articles


In [0]:
# Keywords to match issues to tips
ISSUE_KEYWORDS = {
    "wifi_issues": ["wifi", "internet", "connection", "network"],
    "dirty_general": ["clean", "declutter", "tidy", "spotless", "cleaning"],
    "dirty_bathroom": ["bathroom", "toilet", "shower", "towel", "mirror"],
    "dirty_kitchen": ["kitchen", "countertop", "dishes", "cooking", "utensils"],
    "dirty_bedding": ["sheet", "bed", "pillow", "linen", "blanket", "bedding"],
    "bugs_pests": ["pest", "clean", "hygiene", "mold"],
    "smell_issues": ["smell", "odor", "fresh", "ventilation", "air"],
    "noise_general": ["noise", "quiet", "sound", "peaceful"],
    "noise_street": ["noise", "street", "location", "neighborhood"],
    "noise_neighbors": ["noise", "wall", "neighbor", "sound"],
    "too_hot": ["ac", "air conditioning", "cool", "temperature", "fan"],
    "too_cold": ["heat", "heating", "warm", "temperature", "blanket"],
    "uncomfortable_bed": ["bed", "mattress", "pillow", "sleep", "comfortable"],
    "hot_water_issues": ["water", "hot water", "shower", "plumbing"],
    "plumbing_issues": ["plumbing", "water", "toilet", "drain", "leak"],
    "checkin_problems": ["check-in", "checkin", "arrival", "access", "instructions"],
    "hard_to_find": ["find", "direction", "address", "location", "instructions", "map"],
    "key_issues": ["key", "lock", "lockbox", "access", "entry", "door"],
    "host_unresponsive": ["response", "respond", "communication", "message", "contact"],
    "overpriced": ["price", "pricing", "value", "discount", "competitive"],
    "too_small": ["space", "size", "photo", "accurate", "description"],
    "safety_concerns": ["safe", "safety", "security", "lock", "secure"],
}

def find_relevant_tips(issue, all_tips, max_tips=2):
    """Find tips from knowledge base that match an issue"""
    if issue not in ISSUE_KEYWORDS:
        return []
    
    keywords = ISSUE_KEYWORDS[issue]
    relevant_tips = []
    
    for tip_data in all_tips:
        tip_lower = tip_data['tip'].lower()
        match_count = sum(1 for kw in keywords if kw in tip_lower)
        if match_count >= 1:
            relevant_tips.append({
                'tip': tip_data['tip'],
                'source': tip_data['source'],
                'match_score': match_count
            })
    
    # Sort by match score and return top tips
    relevant_tips.sort(key=lambda x: x['match_score'], reverse=True)
    return relevant_tips[:max_tips]

# Build mapping
print("🔗 Building issue-to-tips mapping...")
issue_to_tips = {}
for issue in ISSUE_KEYWORDS.keys():
    issue_to_tips[issue] = find_relevant_tips(issue, all_tips, max_tips=2)

print(f"✅ Mapped {len(issue_to_tips)} issues to relevant tips")

# Preview
print("\n📋 Sample mappings:")
for issue in ["dirty_general", "wifi_issues", "noise_general"]:
    if issue_to_tips[issue]:
        print(f"\n   {issue}:")
        print(f"      → {issue_to_tips[issue][0]['tip'][:80]}...")

🔗 Building issue-to-tips mapping...
✅ Mapped 22 issues to relevant tips

📋 Sample mappings:

   dirty_general:
      → Clean and declutter. Show guests exactly what they can expect. “It’s surprising ...

   wifi_issues:
      → Wifi: Provide the wifi network and password at your home or location....

   noise_general:
      → Include a short, descriptive listing title. Check that your title offers a uniqu...


In [0]:
# ============================================
# FIXED AGGREGATION - HANDLES NEGATIVE WITHOUT THEMES
# ============================================
ISSUE_TO_TIP_MAPPING = {}

for issue, tips_list in issue_to_tips.items():
    if tips_list:  # If we found tips for this issue
        ISSUE_TO_TIP_MAPPING[issue] = {
            'tip': tips_list[0]['tip'],
            'source': tips_list[0]['source']
        }

print(f"✅ Created ISSUE_TO_TIP_MAPPING with {len(ISSUE_TO_TIP_MAPPING)} issues")
print("📊 Aggregating data by PROPERTY...")
property_data = []

for property_id, group in tqdm(reviews_exploded.groupby('property_id'), desc="Processing properties"):
    
    # ===== BASIC INFO =====
    host_id = group['host_id'].iloc[0]
    listing_name = group['listing_name'].iloc[0]
    location = group['location'].iloc[0]
    total_reviews = len(group)
    
    # ===== SENTIMENT STATS =====
    avg_sentiment = group['sentiment_score'].mean()
    positive_count = (group['sentiment_label'] == 'POSITIVE').sum()
    negative_count = (group['sentiment_label'] == 'NEGATIVE').sum()
    
    # ===== AIRBNB RATINGS =====
    cleanliness_rating = group['cleanliness_rating'].iloc[0]
    communication_rating = group['communication_rating'].iloc[0]
    checkin_rating = group['checkin_rating'].iloc[0]
    location_rating = group['location_rating'].iloc[0]
    value_rating = group['value_rating'].iloc[0]
    
    # ===== THEME COUNTS FOR THIS PROPERTY =====
    all_positive = [t for themes in group['positive_themes'] for t in themes]
    all_negative = [t for themes in group['negative_themes'] for t in themes]
    positive_counts = Counter(all_positive)
    negative_counts = Counter(all_negative)
    
    # ===== FORMAT ADVANTAGES (Top 5) =====
    advantages = []
    for theme, count in positive_counts.most_common(5):
        if count >= 1:
            advantages.append(f"{theme.replace('_', ' ').title()} ({count}x)")
    advantages_str = "; ".join(advantages) if advantages else "No specific themes identified"
    
    # ===== FORMAT DISADVANTAGES (Top 5) =====
    disadvantages = []
    for theme, count in negative_counts.most_common(5):
        if count >= 1:
            disadvantages.append(f"{theme.replace('_', ' ').title()} ({count}x)")
    disadvantages_str = "; ".join(disadvantages) if disadvantages else "No issues identified"
    
    # ===== GENERATE SUGGESTIONS =====
    suggestions = []
    sources = set()
    
    # Get top 3 issues for THIS PROPERTY
    sorted_issues = sorted(negative_counts.items(), key=lambda x: x[1], reverse=True)[:3]
    
    for issue, count in sorted_issues:
        if issue in ISSUE_TO_TIP_MAPPING:
            tip_data = ISSUE_TO_TIP_MAPPING[issue]
            tip_text = ' '.join(tip_data['tip'].split())
            issue_name = issue.replace('_', ' ').title()
            suggestions.append(f"[{issue_name}]: {tip_text}")
            sources.add(tip_data['source'][:70])
    
    # ===== FIXED LOGIC FOR SUGGESTIONS STRING =====
    if suggestions:
        # We found specific issues with suggestions
        suggestions_str = " || ".join(suggestions)
    elif negative_count > 0:
        # There are negative reviews but no specific themes detected
        suggestions_str = f"Note: {negative_count} negative review(s) detected but no specific issues were identified by our analysis. We recommend manually reviewing the feedback for insights."
    else:
        # No negative reviews at all
        suggestions_str = "No issues identified - keep up the great work!"
    
    sources_str = " | ".join(sources) if sources else ""
    
    # ===== STORE PROPERTY DATA =====
    property_data.append({
        'property_id': property_id,
        'host_id': host_id,
        'listing_name': listing_name,
        'location': location,
        'total_reviews': total_reviews,
        'positive_reviews': positive_count,
        'negative_reviews': negative_count,
        'avg_sentiment_score': round(avg_sentiment, 3),
        'cleanliness_rating': cleanliness_rating,
        'communication_rating': communication_rating,
        'checkin_rating': checkin_rating,
        'location_rating': location_rating,
        'value_rating': value_rating,
        'advantages': advantages_str,
        'disadvantages': disadvantages_str,
        'suggestions': suggestions_str,
        'suggestion_sources': sources_str
    })

# Create DataFrame
property_summary = pd.DataFrame(property_data)
property_summary = property_summary.sort_values('total_reviews', ascending=False).reset_index(drop=True)

print(f"\n✅ Aggregated data for {len(property_summary):,} properties")
print(f"   From {property_summary['host_id'].nunique():,} unique hosts")


✅ Created ISSUE_TO_TIP_MAPPING with 22 issues
📊 Aggregating data by PROPERTY...


Processing properties: 100%|██████████| 2246/2246 [00:02<00:00, 1080.86it/s]


✅ Aggregated data for 2,246 properties
   From 1,884 unique hosts


In [0]:

print("=" * 80)
print("📋 SAMPLE PROPERTY REPORTS")
print("=" * 80)

for i, row in property_summary.head(3).iterrows():
    print(f"\n🏠 PROPERTY ID: {row['property_id']}")
    print(f"👤 HOST ID: {row['host_id']}")
    print(f"🏷️  Name: {row['listing_name'][:60]}...")
    print(f"📍 Location: {row['location']}")
    print(f"📊 Reviews: {row['total_reviews']} ({row['positive_reviews']} positive, {row['negative_reviews']} negative)")
    print(f"⭐ Avg Sentiment: {row['avg_sentiment_score']}")
    print(f"\n✅ ADVANTAGES:\n   {row['advantages']}")
    print(f"\n❌ DISADVANTAGES:\n   {row['disadvantages']}")
    print(f"\n💡 SUGGESTIONS:\n   {row['suggestions'][:500]}...")
    print(f"\n📚 Sources: {row['suggestion_sources']}")
    print("-" * 80)

📋 SAMPLE PROPERTY REPORTS

🏠 PROPERTY ID: 6943815
👤 HOST ID: 21271697
🏷️  Name: Entire rental unit in Paris, France...
📍 Location: Paris, Île-de-France, France
📊 Reviews: 119 (112 positive, 7 negative)
⭐ Avg Sentiment: 0.884

✅ ADVANTAGES:
   Great Location (36x); Responsive Host (28x); Highly Recommend (17x); Very Comfortable (13x); Walkable (13x)

❌ DISADVANTAGES:
   Smell Issues (3x); Dirty General (2x); Noise General (2x)

💡 SUGGESTIONS:
   [Smell Issues]: Everyone has a different sense of smell. Strong scents of any kind can be off-putting to guests. Trying to mask odors with bleach, air fresheners, or other solutions usually doesn’t work, Diana says. || [Dirty General]: Clean and declutter. Show guests exactly what they can expect. “It’s surprising how many people forget to clear their spaces of random objects before taking photos,” says Jeff, a photographer and Superhost in Joshua Tree, California. || [Noise General]: Include a ...

📚 Sources: How to become a Superhost - Resourc

In [0]:
# Prepare final DataFrame with clean column names
final_csv = property_summary[[
    'property_id',
    'host_id',
    'listing_name',
    'location',
    'total_reviews',
    'positive_reviews',
    'negative_reviews',
    'avg_sentiment_score',
    'cleanliness_rating',
    'communication_rating',
    'checkin_rating',
    'location_rating',
    'value_rating',
    'advantages',
    'disadvantages',
    'suggestions',
    'suggestion_sources'
]].copy()

# Rename columns for clarity
final_csv.columns = [
    'Property_ID',
    'Host_ID',
    'Listing_Name',
    'Location',
    'Total_Reviews',
    'Positive_Reviews',
    'Negative_Reviews',
    'Avg_Sentiment_Score',
    'Cleanliness_Rating',
    'Communication_Rating',
    'Checkin_Rating',
    'Location_Rating',
    'Value_Rating',
    'Advantages',
    'Disadvantages',
    'Suggestions',
    'Suggestion_Sources'
]

# Export to CSV
output_filename = 'property_review_intelligence_report.csv'
final_csv.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"✅ CSV EXPORTED: {output_filename}")
print(f"\n📊 SUMMARY:")
print(f"   Total properties: {len(final_csv):,}")
print(f"   Unique hosts: {final_csv['Host_ID'].nunique():,}")
print(f"   Total reviews analyzed: {final_csv['Total_Reviews'].sum():,}")

print(f"\n📋 COLUMNS:")
for col in final_csv.columns:
    print(f"   • {col}")

print(f"\n📋 FIRST 5 ROWS:")
display(final_csv.head())

✅ CSV EXPORTED: property_review_intelligence_report.csv

📊 SUMMARY:
   Total properties: 2,246
   Unique hosts: 1,884
   Total reviews analyzed: 41,225

📋 COLUMNS:
   • Property_ID
   • Host_ID
   • Listing_Name
   • Location
   • Total_Reviews
   • Positive_Reviews
   • Negative_Reviews
   • Avg_Sentiment_Score
   • Cleanliness_Rating
   • Communication_Rating
   • Checkin_Rating
   • Location_Rating
   • Value_Rating
   • Advantages
   • Disadvantages
   • Suggestions
   • Suggestion_Sources

📋 FIRST 5 ROWS:


Property_ID,Host_ID,Listing_Name,Location,Total_Reviews,Positive_Reviews,Negative_Reviews,Avg_Sentiment_Score,Cleanliness_Rating,Communication_Rating,Checkin_Rating,Location_Rating,Value_Rating,Advantages,Disadvantages,Suggestions,Suggestion_Sources
6943815,21271697,"Entire rental unit in Paris, France","Paris, Île-de-France, France",119,112,7,0.884,4.8,4.9,4.9,4.9,4.7,Great Location (36x); Responsive Host (28x); Highly Recommend (17x); Very Comfortable (13x); Walkable (13x),Smell Issues (3x); Dirty General (2x); Noise General (2x),"[Smell Issues]: Everyone has a different sense of smell. Strong scents of any kind can be off-putting to guests. Trying to mask odors with bleach, air fresheners, or other solutions usually doesn’t work, Diana says. || [Dirty General]: Clean and declutter. Show guests exactly what they can expect. “It’s surprising how many people forget to clear their spaces of random objects before taking photos,” says Jeff, a photographer and Superhost in Joshua Tree, California. || [Noise General]: Include a short, descriptive listing title. Check that your title offers a unique, accurate perspective of your home and surroundings. For example, “Quiet studio near Uffizi” conveys in a few words that the place is a peaceful, centrally located apartment in Florence, Italy.",How to become a Superhost - Resource Center - Airbnb | How to take great photos for your listing - Resource Center - Airbnb | Clean like a pro with these expert tips - Resource Center - Airbnb
4721885,6768004,"Entire rental unit in Paris, France","Paris, Île-de-France, France",72,69,3,0.916,4.9,5.0,5.0,5.0,4.8,Great Location (27x); Very Comfortable (17x); Responsive Host (17x); Highly Recommend (14x); Will Return (11x),No issues identified,Note: 3 negative review(s) detected but no specific issues were identified by our analysis. We recommend manually reviewing the feedback for insights.,
989255591588417567,37529970,"Entire townhouse in Paris, France","Paris, Île-de-France, France",72,72,0,1.0,5.0,5.0,5.0,5.0,4.9,Beautiful Place (25x); Walkable (16x); Responsive Host (16x); Highly Recommend (14x); Great Host (11x),No issues identified,No issues identified - keep up the great work!,
932572006139750529,14469088,"Entire rental unit in Paris, France","Paris, Île-de-France, France",72,72,0,1.0,5.0,5.0,5.0,4.9,4.9,Spacious (15x); Will Return (15x); Very Clean (12x); Very Comfortable (11x); Beautiful Place (9x),No issues identified,No issues identified - keep up the great work!,
1596929,7642792,"Entire rental unit in Paris, France","Paris, Paris province, France",72,57,15,0.587,4.5,4.7,4.7,4.9,4.4,Great Location (22x); Walkable (9x); Quiet Area (9x); Responsive Host (9x); Very Comfortable (8x),No issues identified,Note: 15 negative review(s) detected but no specific issues were identified by our analysis. We recommend manually reviewing the feedback for insights.,
